**lib**

In [54]:
def parse_tf_example(tf_example):
    parsed_example = tf.io.parse_single_example(tf_example,
                                               {
                                                   'image': tf.io.FixedLenFeature((), tf.string),
                                                   'dimension': tf.io.VarLenFeature(tf.float32),
                                                   'orientation': tf.io.VarLenFeature(tf.float32),
                                                   'confidence': tf.io.VarLenFeature(tf.float32)
                                               })
    return {"image": parsed_example['image'],
            'dimension': parsed_example['dimension'],
            'orientation': parsed_example['orientation'],
            'confidence': parsed_example['confidence']}

In [55]:
import tensorflow as tf
import matplotlib.pyplot as plt

train_file = "/Users/privy/Desktop/all_repos/3D_bb_estimation/data/train.tfrecords"
number_bin = 2
batch_size = 32
shuffle_buffer_size = 10000

def preprocess(image_dataset):
    # parse image
    image = tf.io.decode_image(image_dataset["image"])
    image = tf.divide(image, 255)
    # parse dimension
    dimension = image_dataset["dimension"].values
    # parse and reshape orientation
    orientation = image_dataset["orientation"].values
    orientation = tf.reshape(orientation, (number_bin, 2))
    # parse confidence
    confidence = image_dataset["confidence"].values
    return image, dimension, orientation, confidence

def generate_dataset(file_path):
    dataset = tf.data.TFRecordDataset(file_path)
    dataset = dataset.map(parse_tf_example)
    dataset = dataset.map(preprocess)
    dataset = dataset.shuffle(shuffle_buffer_size, seed=12)
    # split to train and val
    dataset_size = len(list(dataset))
    train_size = int(dataset_size*0.8)
    train_dataset = dataset.take(train_size)
    val_dataset = dataset.skip(train_size)
    # create batch
    train_dataset = train_dataset.batch(batch_size)
    val_dataset = val_dataset.batch(batch_size)
    return train_dataset, val_dataset

train_dataset, val_dataset = generate_dataset(train_file)

In [66]:
from tensorflow.keras.applications import VGG16

base_model = VGG16(include_top=False, weights="imagenet", input_shape=(224, 224, 3))

In [67]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     